### Import dependencies :

In [72]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
import datetime
from sklearn.preprocessing import StandardScaler , OneHotEncoder
import joblib


### Load Data :

In [73]:
data=pd.read_csv('no_outliers.csv')
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,0.946187,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,429.332315,0.133511
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,0.910936,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,194.210750,0.363279
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,0.896924,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,648.404343,0.280041
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,0.869940,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,885.138727,0.125099
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,0.878946,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,602.697524,0.320636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,0.668902,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,534.140819,0.364438
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,0.818049,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,387.264753,0.564494
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,0.896188,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,476.450471,0.591383
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,0.860886,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1015.811664,-0.093105


In [76]:
data['BOA_fraction']=data['BOA_RT']/1000
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha,BOA_fraction
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,...,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,429.332315,0.133511,0.429332
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,...,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,194.210750,0.363279,0.194211
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,...,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,648.404343,0.280041,0.648404
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,...,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,885.138727,0.125099,0.885139
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,...,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,602.697524,0.320636,0.602698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,...,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,534.140819,0.364438,0.534141
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,...,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,387.264753,0.564494,0.387265
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,...,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,476.450471,0.591383,0.476450
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,...,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1015.811664,-0.093105,1.015812


### Preprocessing :

In [77]:
from sklearn.preprocessing import StandardScaler 

In [78]:
BOA_RT=data['BOA_RT']
data.drop(columns=['BOA_RT'],inplace=True)

In [79]:
data.drop(columns=['alpha'],inplace=True)
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,0.946187,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,0.429332
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,0.910936,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,0.194211
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,0.896924,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,0.648404
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,0.869940,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,0.885139
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,0.878946,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,0.602698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,0.668902,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,0.534141
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,0.818049,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,0.387265
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,0.896188,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,0.476450
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,0.860886,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1.015812


In [80]:
scaler=StandardScaler()
scaler_y=StandardScaler()
cols_to_scale = data.drop(columns=['BOA_fraction']).columns
X_scaled = scaler.fit_transform(data[cols_to_scale])
y_scaled=scaler_y.fit_transform(data['BOA_fraction'].values.reshape(-1, 1))

In [81]:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y_scaled,test_size=0.25,random_state=42)

In [82]:
with open("best_params.json", "r") as f:
    params = json.load(f)

# Build the model
model = Sequential()

# Layer 1
model.add(Dense(params["units1"], activation=params["activation"], input_shape=(X_train.shape[1],)))
model.add(Dropout(params["dropout1"]))

# Layer 2
model.add(Dense(params["units2"], activation=params["activation"]))
model.add(Dropout(params["dropout2"]))

# Layer 3 (if applicable)
if params["n_layers"] >= 3:
    model.add(Dense(params["units3"], activation=params["activation"]))
    model.add(Dropout(params["dropout3"]))

# Layer 4 (if applicable)
if params["n_layers"] == 4:
    model.add(Dense(params["units4"], activation=params["activation"]))
    model.add(Dropout(params["dropout4"]))

# Output layer
model.add(Dense(1))

# Compile model
optimizer = Adam(learning_rate=params["lr"])
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class TransmissionLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        alpha, m_g, m_a, dtau_gas_scat, dtau_aero_scat, T_gas_abs, T_aero_abs = inputs

        # Dénominateur (1 + α⋅Δτ_gas⋅m_g + α/3⋅Δτ_aero⋅m_a)
        denom = 1 + alpha * dtau_gas_scat * m_g + (alpha / 3.0) * dtau_aero_scat * m_a

        # Absorptions
        T_abs = tf.pow(T_gas_abs, m_g) * tf.pow(T_aero_abs, m_a)

        # E_BOA / E_TOA
        E_BOA_div_TOA = (T_abs/ denom) 
        return E_BOA_div_TOA

In [47]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout

# Entrée principale : 18 descripteurs
features_input = Input(shape=(18,), name='features')

# Autres entrées nécessaires à la formule analytique
m_g_input = Input(shape=(1,), name='mprime_g')
m_a_input = Input(shape=(1,), name='mprime_a')
dtau_gas_scat_input = Input(shape=(1,), name='GOD')
dtau_aero_scat_input = Input(shape=(1,), name='AODS')
T_gas_abs_input = Input(shape=(1,), name='Tg_abs')
T_aero_abs_input = Input(shape=(1,), name='Ta_abs')

In [48]:
x = Dense(232, activation='elu')(features_input)
x = Dropout(0.111)(x)
x = Dense(249, activation='elu')(x)
x = Dropout(0.138)(x)
x = Dense(174, activation='elu')(x)
x = Dropout(0.136)(x)

# Prédiction de alpha
alpha_output = Dense(1, name='alpha')(x)


In [49]:
E_BOA_div_TOA = TransmissionLayer()([
    alpha_output,
    m_g_input,
    m_a_input,
    dtau_gas_scat_input,
    dtau_aero_scat_input,
    T_gas_abs_input,
    T_aero_abs_input
])
model = Model(
    inputs=[features_input, m_g_input, m_a_input, dtau_gas_scat_input, dtau_aero_scat_input, T_gas_abs_input, T_aero_abs_input],
    outputs=E_BOA_div_TOA
)

In [50]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00034),
    loss='mae',  # ou 'mae' selon ce que tu veux
    metrics=['mae']
)

In [51]:
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,0.946187,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,0.429332
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,0.910936,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,0.194211
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,0.896924,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,0.648404
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,0.869940,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,0.885139
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,0.878946,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,0.602698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,0.668902,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,0.534141
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,0.818049,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,0.387265
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,0.896188,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,0.476450
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,0.860886,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1.015812


In [52]:
X_features = data.iloc[:, :18].values

# autres colonnes utilisées dans la couche analytique
X_inputs = {
    'features': X_features,
    'mprime_g': data['mprime_g'].values.reshape(-1, 1),
    'mprime_a': data['mprime_a'].values.reshape(-1, 1),
    'GOD': data['GOD'].values.reshape(-1, 1),
    'AODS': data['AODS'].values.reshape(-1, 1),
    'Tg_abs': data['Tg_abs'].values.reshape(-1, 1),
    'Ta_abs': data['Ta_abs'].values.reshape(-1, 1),
}

# y = valeur cible (E_BOA / E_TOA) → entre 0 et 1
y = data['BOA_fraction'].values

# entraînement


In [53]:
# 1. Early stopping
early_stop = EarlyStopping(
    monitor='val_mae',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# 2. Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# 3. Model checkpoint (save best model)
checkpoint_cb = ModelCheckpoint(
    filepath='deep_model_1.keras',
    monitor='val_mae',
    save_best_only=True,
    verbose=1
)

# 4. CSV logger (log training history)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_logger = CSVLogger(f'training_log_{timestamp}.csv')

In [54]:
model.fit(
    X_inputs,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr, checkpoint_cb, csv_logger]
)

Epoch 1/100
4524/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5624 - mae: 0.5624
Epoch 1: val_mae improved from inf to 0.55872, saving model to deep_model_1.keras
4532/4532 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - loss: 0.5624 - mae: 0.5624 - val_loss: 0.5587 - val_mae: 0.5587 - learning_rate: 3.4000e-04
Epoch 2/100
4523/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5572 - mae: 0.5572
Epoch 2: val_mae improved from 0.55872 to 0.55869, saving model to deep_model_1.keras
4532/4532 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 0.5572 - mae: 0.5572 - val_loss: 0.5587 - val_mae: 0.5587 - learning_rate: 3.4000e-04
Epoch 3/100
4521/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5579 - mae: 0.5579
Epoch 3: val_mae improved from 0.55869 to 0.55869, saving model to deep_model_1.keras
4532/4532 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 0.5579 - mae: 0.5579 - val_loss: 0.5587 - val_mae: 0.5587 - learning_rate: 3.4000e-04
Epoch 4/100
4525/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5572 - mae: 0.5

In [68]:
import tensorflow as tf
import json
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda

# Chargement des paramètres optimaux
with open("best_params.json", "r") as f:
    params = json.load(f)

# Fonction d'activation personnalisée qui ne reçoit les 6 paramètres qu'en sortie
def transmission_activation(inputs):
    alpha, m_g, m_a, dtau_gas_scat, dtau_aero_scat, T_gas_abs, T_aero_abs = inputs  # Décomposer la liste d'inputs
    
    # Dénominateur
    denom = 1 + alpha * dtau_gas_scat * m_g + (alpha / 3.0) * dtau_aero_scat * m_a
    
    # Absorption
    T_abs = tf.pow(T_gas_abs, m_g) * tf.pow(T_aero_abs, m_a)
    
    # Transmission finale
    return T_abs / denom

# Définition des entrées complètes
input_features = Input(shape=(X_train.shape[1],))  # Inclut toutes les colonnes pour l'entraînement

# Couches cachées
hidden = Dense(params["units1"], activation=params["activation"])(input_features)
hidden = Dropout(params["dropout1"])(hidden)

hidden = Dense(params["units2"], activation=params["activation"])(hidden)
hidden = Dropout(params["dropout2"])(hidden)

if params["n_layers"] >= 3:
    hidden = Dense(params["units3"], activation=params["activation"])(hidden)
    hidden = Dropout(params["dropout3"])(hidden)

if params["n_layers"] == 4:
    hidden = Dense(params["units4"], activation=params["activation"])(hidden)
    hidden = Dropout(params["dropout4"])(hidden)

# Transmission directe de la sortie avant activation
alpha = Lambda(lambda x: x)(hidden)  # Passage direct sans transformation

# Entrées des 6 paramètres après entraînement
input_m_g = Input(shape=(1,))
input_m_a = Input(shape=(1,))
input_dtau_gas_scat = Input(shape=(1,))
input_dtau_aero_scat = Input(shape=(1,))
input_T_gas_abs = Input(shape=(1,))
input_T_aero_abs = Input(shape=(1,))

# Application de la fonction analytique sur la sortie après entraînement
# Application de la fonction analytique sur la sortie
output = Lambda(lambda inputs: transmission_activation(inputs), output_shape=(None, 1))(
    [alpha, input_m_g, input_m_a, input_dtau_gas_scat, input_dtau_aero_scat, input_T_gas_abs, input_T_aero_abs]
)

# Définition du modèle final
model = Model(inputs=[input_features, input_m_g, input_m_a, input_dtau_gas_scat, input_dtau_aero_scat, input_T_gas_abs, input_T_aero_abs], outputs=output)

# Compilation du modèle
optimizer = tf.keras.optimizers.Adam(learning_rate=params["lr"])
model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

# Affichage du résumé du modèle
model.summary()


Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_31      │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 232)       │      4,408 │ input_layer_31[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 232)       │          0 │ dense_33[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 249)       │     58,017 │ dropout_30[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 249)       │          0 │ dense_34[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 174)       │     43,500 │ dropout_31[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_32          │ (None, 174)       │          0 │ dense_35[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_9 (Lambda)   │ (None, 174)       │          0 │ dropout_32[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_32      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_33      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_34      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_35      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_36      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_37      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_10 (Lambda)  │ (None, None, 1)   │          0 │ lambda_9[0][0],   │
│                     │                   │            │ input_layer_32[0… │
│                     │                   │            │ input_layer_33[0… │
│                     │                   │            │ input_layer_34[0… │
│                     │                   │            │ input_layer_35[0… │
│                     │                   │            │ input_layer_36[0… │
│                     │                   │            │ input_layer_37[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 105,925 (413.77 KB)

 Trainable params: 105,925 (413.77 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
data
X_train=data.drop(columns=['BOA_fraction'])
y_train=data['BOA_fraction']
y_train.shape

(362503,)

In [70]:
X_train_m_g = X_train["mprime_g"].values.reshape(-1, 1)
X_train_m_a = X_train["mprime_a"].values.reshape(-1, 1)
X_train_dtau_gas_scat = X_train["GOD"].values.reshape(-1, 1)
X_train_dtau_aero_scat = X_train["AODS"].values.reshape(-1, 1)
X_train_T_gas_abs = X_train["Tg_abs"].values.reshape(-1, 1)
X_train_T_aero_abs = X_train["Ta_abs"].values.reshape(-1, 1)


In [71]:
history = model.fit(
    [X_train, X_train_m_g, X_train_m_a, X_train_dtau_gas_scat, X_train_dtau_aero_scat, X_train_T_gas_abs, X_train_T_aero_abs], 
    y_train,
    batch_size=32,
    epochs=100
)


Epoch 1/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 1.5567 - mae: 1.5567
Epoch 2/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 1.6487 - mae: 1.6487
Epoch 3/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 1.7287 - mae: 1.7287
Epoch 4/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - loss: 1.9605 - mae: 1.9605
Epoch 5/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 2.0148 - mae: 2.0148
Epoch 6/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - loss: 2.1538 - mae: 2.1538
Epoch 7/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - loss: 2.0942 - mae: 2.0942
Epoch 8/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 2.1925 - mae: 2.1925
Epoch 9/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - loss: 2.1599 - mae: 2.1599
Epoch 10/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 2.3423 - mae: 2.3423
Epoch 11/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - loss: 2.1829 - mae: 2.1829
Epoch 12/100
11329/11329 ━━━━━━━━━━━━━━━━

KeyboardInterrupt: 